# Análisis de variantes desde lecturas de secuenciador masivo (v.2023)

> Jorge Maturana, Instituto de Informática UACh, jorge.maturana@uach.cl


### Presentación

Este tutorial presenta un pipeline de análisis de datos obtenidos desde un secuenciador masivo de datos paired-end, hasta llegar a la identificación de variantes. Para ello se propone un pieline basado en línea de comandos de Linux, con herramientas instaladas en un ambiente de Conda

### Presentación

Este tutorial se estructura de la siguiente forma:

1. Seteo del ambiente
 - Conda
 - Jupyter Notebook
 - Creación de estructura de directorios
 - Descarga de DNA de referencia
 - Obtención de archivos desde el secuenciador
1. Control de calidad
1. Trimming
1. Alineamiento de datos podados pareados
 - Indexación de ADN de referencia
 - Alineamiento
1. Variant calling
 - Creación de archivo pileup
 - Llamado de variantes



## 1. Seteo del ambiente

### Conda

**Conda** es un administrador de paquetes multiplataforma que permite generar ambientes destinados a distintas actividades en el mismo computador. Para más información sobre conda visite [este link](https://docs.conda.io/en/latest/).

**Miniconda** es una implementación mínima de Conda. Una introducción breve y las instrucciones para la instalación pueden encontrarse en [este video](https://www.youtube.com/watch?v=23aQdrS58e0).

En este tutorial se utilizará Conda. Se reecomienda crear un ambiente llamado *bio* con Python 3.9:

In [ ]:
!conda create -n bio python=3.9

Activamos el ambiente bio de conda

In [ ]:
!conda init bash

Agregamos el canal **bioconda**, que contiene varios de los programas que utilizaremos

In [ ]:
!conda config --add channels bioconda

Instalaremos algunos paquetes que nos serán de utilidad:

In [ ]:
!conda install -c bioconda fastqc # para analizar la calidad de los reads
!conda install -c bioconda bwa # para alinear los reads contrar un genoma de referencia
!conda install -c bioconda samtools # para transformar el archivo de alieamiento en un formato desplegable de manera gráfica
!conda install -c bioconda igv # para visualizar gráficamente los aineamientos
!conda install -c bioconda bcftools # para identificar las variantes (mutaciones)
!conda install -c conda-forge py-bgzip # para comprimir en DNAref previo al pileup del llamado de variantes

### Jupyter Notebook

Jupyter Notebook es un ambiente basado en web para el desarrollo interactivo de codigo y texto. Una introducción rápida y simple se entrega en [este video](https://www.youtube.com/watch?v=6Vr9ZUntCyE).

### Creación de estructura de directorios

Durante el proceso de análisis se generan muchos archivos. Crear una estructura de directorios es conveniente porque:
- Permite proteger los datos crudos de eliminaciones o modificaciones accidentales
- Permite separar los archivos generados en cada paso del proceso, en caso de rollback



In [ ]:
!mkdir RawReads # para los reads crudos (que vienen del secuenciador)
!mkdir DNAref # para almacenar el DNA de referencia
!mkdir FastQC_RawData # para guardar los resultados de calidad de los reads crudos
!mkdir TrimReads # para almacenar los reads podados
!mkdir FastQC_TrimData # para guardar los resultados de calidad de los reads podados
!mkdir Align # para almacenar los resultados del alineamiento
!mkdir VarCall # para almacenar las variantes llamadas

### Descarga de DNA de referencia

Es necesario contar con el ADN de referencia para alinear los reads. Los reads que analizaremos proceden del cromosoma X de la mosca de la fruta (*Drosophila melanogaster*) 


<img src="images_JupNot/Drosophila_melanogaster.jpg" alt="Mosca de la fruta (Drosophila melanogaster)" style="width: 400px;"/>

Este se puede obtener desde una base de datos. En nuestro caso, se cuenta con el link de descarga directo desde NCBI. Descargaremos el archivo mediante la utilidad **wget** y la almacenaremos en el directorio DNAref. 

In [ ]:
%cd DNAref
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/215/GCF_000001215.4_Release_6_plus_ISO1_MT/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.gz
%cd ..

### Obtención de archivos desde el secuenciador

Los archivos desde el secuenciador vienen en formato FASTQ. En este tutorial utilizaremos dos archivos procedentes de una secuenciación Paired-end de Drosophila melanogaster, los cuales fueron utilizados originalmente en el tutorial de Bérénice Batut que se puede encontrar [acá](https://training.galaxyproject.org/archive/2021-08-01/topics/sequence-analysis/tutorials/quality-control/tutorial.html#process-paired-end-data)


In [ ]:
%cd RawReads/
# descarga de datos
!wget https://zenodo.org/record/61771/files/GSM461178_untreat_paired_subset_1.fastq
!wget https://zenodo.org/record/61771/files/GSM461178_untreat_paired_subset_2.fastq
%cd ..

Los nombres de archivos de genoma pueden ser muy largos, lo que podría confundirnos al momento de tipear los comandos. Es posible trabajar localmente con nombres más simples

Para no perder el nombre de los archivos y no utilizar más espacio de disco que el necesario, se pueden usar links simbólicos de Linux (similares a los accesos directos en Windows), los cuales crean un puntero a otro archivo

In [ ]:
%cd RawReads/
# creación de links simbólicos
!ln -s GSM461178_untreat_paired_subset_1.fastq R1.fq
!ln -s GSM461178_untreat_paired_subset_2.fastq R2.fq
%cd ..

## 2. Control de calidad

### análisis de muestras crudas con FastQC

Para observar la calidad de los reads crudos se puede invocar FastQC

In [ ]:
!fastqc RawReads/R1.fq -o FastQC_RawData/
!fastqc RawReads/R2.fq -o FastQC_RawData/

### Analizar Reportes de FastQC

FastQC genera una página web con distintos módulos de análisis en los que se despliegan estadísticas de los datos. Las siguientes gráficas muestran las calidades por base de los reads para [R1](./FastQC_RawData/R1_fastqc.html) y [R2](./FastQC_RawData/R2_fastqc.html):

<table><tr>
<td> <img src="images_JupNot/FastQC_rawR1.png" alt="FastQC raw R1" style="width: 400px;"/> </td>
<td> <img src="images_JupNot/FastQC_rawR2.png" alt="FastQC raw R2" style="width: 400px;"/> </td>
</tr></table>


## 3. Trimming

El proceso de **trimming** (o poda) permite eliminar reads de baja calidad y adaptadores sintéticos que pudieran haberse secuenciado por error.

Al realizar una poda de dos archivos (FASTQ) con reads paired-end, se generan 4 archivos FASTQ con reads pareados (cuyo par en el sentido contrario de lectura ha sobrevivido), y con reads no-pareados para cada una de las lecturas, R1 y R2.

<img src="images_JupNot/trimming.png" alt="Trimming" style="width: 700px;"/>

La documetación de trimmomatic puede encontrse [aquí](http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/TrimmomaticManual_V0.32.pdf).

A continuación se ejecutará trimmomatic sobre los datos Paired End que se encuentran en el directorio RawReads. Los resultados se almacenarán en el directorio TrimReads

In [ ]:
!trimmomatic PE RawReads/R1.fq RawReads/R2.fq TrimReads/p_R1.fq TrimReads/u_R1.fq TrimReads/p_R2.fq TrimReads/u_R2.fq LEADING:3 TRAILING:3 SLIDINGWINDOW:4:20 MINLEN:20

   ### FastQC sobre datos podados
   
   Para apreciar el efecto del trimming, se analizan los archivos con los reads podados con FastQC
   

In [ ]:
!for i in `ls TrimReads/*.fq`; do fastqc $i -o FastQC_TrimData/; done

Las siguientes gráficas muestran las calidades por base de los reads podados en los archivos pareados de [R1](./FastQC_TrimData/p_R1_fastqc.html) y [R2](./FastQC_TrimData/p_R2_fastqc.html), las cuales se pueden comparar con la de los datos crudos, mostrados anteriormente

<table><tr>
<td> <img src="images_JupNot/FastQC_trim_pR1.png" alt="FastQC trimmed paired R1" style="width: 400px;"/> </td>
<td> <img src="images_JupNot/FastQC_trim_pR2.png" alt="FastQC trimmed paired R2" style="width: 400px;"/> </td>
</tr></table>


## 4. alineamiento de datos podados pareados

De manera arbitraria, sólo alinearemos los reads de los datos pareados que sobrevivieron a la poda. Este proceso se separa en dos pasos: indexación y alineamiento propiamente tal

<img src="images_JupNot/alineamiento.png" alt="Alineamiento" style="width: 700px;"/>


### Indexación de ADN de referencia

Dado que elproceso de alinear los reads contra el ADN de referencia es trabajoso, es necesaario indexarlo. Este proceso puede tardarse, pero se realiza solamente una vez, dado que es independiente de los reads. Para crear tal ínidce se invoca el método de indexación del alineador bwa (Burrows-Wheeler Aligner)

In [ ]:
!bwa index DNAref/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.gz

### Alineamiento

Una de las herramientas más utilizadas para realizar el alineamiento es ele Burrows-Wheeler Aligner. Este proceso genera un archivo SAM, que será almacenado en la carpeta Align

In [ ]:
!bwa mem DNAref/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.gz TrimReads/p_R1.fq TrimReads/p_R2.fq > Align/trimPaired.sam

###  Visualización con IGV

Nos interesa visualizar el alineamiento para tener una idea de lo que ha resultado de este proceso. Esto se puede hacer con el programa IGV.

Para desplegar los resultados en IGV, se debe transformar el archivo SAM a su equivalente comprimido BAM, y posteriormente indexarlo.

In [ ]:
%cd Align/
!samtools sort trimPaired.sam -o sorted_trimPaired.bam
!samtools index sorted_trimPaired.bam
%cd ..

In [ ]:
!igv Align/sorted_trimPaired.bam

IGV despliega los reads bajo el genoma de referencia. Tanto algunos reads como algunos nucleótidos están marcados en color, para llamar la atención sobre alguna situación especial. El significado de cada color puede ser revisado [acá](https://software.broadinstitute.org/software/igv/AlignmentData).

Debe asegurarse que el ADN de referencia contra el cual se están desplegando los reads es el correcto. Een nuestro caso, **D. melanogaster (dm6)**

A Modo de ejemplo, en el locus **chr2L:10,201,310-10,202,060** se observa una cantidad significativa de reads. Un **locus** es una ubicación dentro del genoma, en nuestro caso en el **cromosoma 2L** entre las posiciones 10,201,310 y 10,202,060

<img src="images_JupNot/IGV_profLectura.png" alt="IGV porfundidad de lectura" style="width: 700px;"/>

En el locus **chr2L:9,967,742-9,968,167** aparecen dos SNPs con bastante claridad (barras verticales azul y roja en la misma posición de varios reads diferentes)

<img src="images_JupNot/IGV_SNPs.png" alt="IGV SNPs" style="width: 700px;"/>


## 5. Variant calling

El llamado de variantes permite distinguir, con una alta probabilidad, las variantes verdaderas de los artefactos producidos durante el proceso de laboratorio (considerando, entre otros, la obtención y preparación de muestras, PCR, preparación de librerías y secuenciación)

Este procedimiento se realiza en dos pasos: creación del archivo **pileup**, y llamado de variantes propiamente tal

<img src="images_JupNot/varCall.png" alt="Llamado de variantes" style="width: 700px;"/>


### Creación de archivo pileup

El proceso de pileup genera un archivo que resume la posición de las variantes encontradas para cada posición en distintos reads. Se realiza con la utilidad *samtools mpileup*, la cual requiere un ADN de referencia descomprimido o bien comprimido con una utilidad similar a *gzip*, llamada *bgzip*

In [ ]:
!gunzip DNAref/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.gz #descomprimir .gz
!bgzip DNAref/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna # volver a comprimir el FASTA con bgzip

In [ ]:
!samtools mpileup Align/sorted_trimPaired.bam -g -f DNAref/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.gz > VarCall/pileup.bcf

### Llamado de variantes

El llamado de variantes propiamente tal se realiza con la herramienta bcftools call

In [ ]:
!bcftools call -O b -vc VarCall/pileup.bcf > VarCall/varCall.bcf
#(opción -O b indica que el output es BCF, -O v sacaría un VCF)

# transformar BCF a VCF
!bcftools view VarCall/varCall.bcf > VarCall/varCall.vcf

### Visualización de llamado de variantes en IGV

Las variantes identificadas pueden visualizarse junto a los reads con la ayuda de IGV

In [ ]:
!igv Align/sorted_trimPaired.bam VarCall/varCall.vcf

> **Esta sección contiene un desvío del pipeline**
> 
> Por alguna razón, al intentar visualizar los llamados de variantees con referencia al genoma y anotaciones de D. melanogaster incluidos en IGV, estos no so desplegados.
> una hipótesis es que los nombres del FASTA utilizado por IGV y el alineamiento no son los mismos

Una forma de verificar lo anterior es explorar los nombres de los encabezados del FASTA del DNA de referencia. Para esto, desplegamos el listado de encabezados, filtrando con el carácter ">"

> **cat** y **grep** son utilidades de la línea de comanndos. cat muestra el contenido de todo el archivo, el resultodo de eso, en vez de ser mostrado por pantalla, es canalizado a través del operador **pipe** (**|**) hacia grep, que filtra las líneas que contienen el carácter ">", dejando fuera a las secuencas y dejando pasar sólo los encabezados

In [ ]:
!gunzip  DNAref/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna.gz #descomprimir DNAref
!cat DNAref/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna | grep ">"

Una gran cantidad de secuencias corresponden a contigs o scaffolds (marcados con el prefijo *NW_*. Para ignorar estos, volvemos a filtrar para eliminar estos encabezados, esta vez utilizando la opción **-v** de grep, que invierte el filtro

In [ ]:
!cat DNAref/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna | grep ">" | grep -v "NW_"

Observamos que en este archivo el cromosoma 2L está identificado como

> \> NT_033779.5 Drosophila melanogaster chromosome 2L

Por lo que el locus en donde se habían observado los SNPs (**chr2L**:9,967,742-9,968,167) aparecerá ahora bajo el nombre **NT_033779.5**:9,967,742-9,968,167) 

Para visualizar las variantes llamadas, debemos utilizar archivos que utilicen los mismos términos en la anotación de los genes. Podemeos hacer esto utilizando el DNAref que se utilizó para hacer el alineamiento y el llamado de variantes (almacenado en el directorio DNAref) en vez del que está cargado en IGV. Para hacer esto haremos lo siguiente:

1. Iniciar IGV
1. En el menú *Genomes -> Load Genomes From File...* cargar el genoma de referencia almacenado en la carpeta DNAref (GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.fna)
1. En el menú *File -> Load From File...* cargar VarCall/varCall.vcf
1. En el menú *File -> Load From File...* cargar Align/trimPaired.bam
1. En la ventana del locus, colocar **NT_033779.5:9,967,742-9,968,167**

Con esto, podemos visualizar que las variantes que se habían observado a simple fueron confirmadas en el proceso de llamado de variantes (No así las variantes menos representativaas en **NT_033779.5:10,201,310-10,202,060**)

<img src="images_JupNot/IGV_varCall.png" alt="IGV llamado de variante" style="width: 700px;"/>

Es posible cargar las anotaciones del genoma de referencia descargando de desde NCBI el archivo GFF (link [acá](https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/215/GCF_000001215.4_Release_6_plus_ISO1_MT/GCF_000001215.4_Release_6_plus_ISO1_MT_genomic.gff.gz)). Al cargar en el menú *File -> Load From File...* se podrá observar que esta variante pertenece al gen RpL13, para el cual se podría buscar mayor información en alguna base de datos especializada como [FlyBase](https://flybase.org/).

<img src="images_JupNot/IGV_varCall_gene_RpL13.png" alt="IGV identificaicón de gen" style="width: 700px;"/>
